<a href="https://colab.research.google.com/github/kangkwanghyup/2022test/blob/main/%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D(%EC%9B%90%EB%8D%B0%EC%9D%B4_%ED%81%B4%EB%A0%88%EC%8A%A4_23_10_18_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [53]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}

In [55]:
response = requests.get("https://news.naver.com/main/ranking/popularMemo.naver", headers=headers)
html=response.text
soup=BeautifulSoup(html,'html.parser')
word=soup.select_one(".rankingnews_tit")

In [56]:
word.text

'언론사별 랭킹뉴스'

In [57]:
newslist = soup.select(".rankingnews_list")

In [58]:
len(newslist)

80

In [ ]:
newslist

In [61]:
newsData = []

for news in newslist[:12]:  #12개의 언론사
  lis = news.find_all("li") #상위랭킹 5개의 뉴스데이터 수집
  for li in lis:
    news_ranking = li.select_one(".list_ranking_num").text     #랭킹
    news_title = li.select_one(".list_title").text             #제목
    news_link = li.select_one(".list_title").get("href")       #링크
    try:                                                       #썸네일
      news_img = li.select_one("img").get("src")
    except:
      news_img = None


    response = requests.get(news_link, headers=headers)       #뉴스본문링크 주소로 get요청
    html=response.text
    soup = BeautifulSoup(html,'html.parser')

    news_time = soup.select_one(".media_end_head_info_datestamp_time._ARTICLE_DATE_TIME").get("data-date-time")
    news_content = soup.select_one("#newsct_article")
    news_content = news_content.text.replace("\n", "").replace("\t", "") if news_content else None           #문자열에서 줄바꿈(\n) 문자열 제거,  탭(\t)문자를 공백 제거

    newsData.append([news_ranking, news_title, news_link, news_img, news_time, news_content])    #내용저장

In [60]:
newsData

[['1',
  '서울대 치대도 포기...무조건 의대로',
  'https://n.news.naver.com/article/052/0001948558?ntype=RANKING',
  'https://mimgnews.pstatic.net/image/origin/052/2023/10/18/1948558.jpg?type=nf70_70',
  '2023-10-18 15:02:12',
  "[앵커]의대 쏠림이 심해지면서 서울대조차 중도 포기자가 늘고 있습니다.이공계는 물론 같은 의약계열 내에서조차 의대를 좇는 현상이 심해지고 있는데 의사 증원과 더불어 인재 양성 전반에 대한 대책이 함께 제시돼야 한다는 지적입니다.김현아 기자가 보도합니다.[기자]어떤 대학이든, 몇 수를 하든, 무조건 의대로!최상위권 학생 사이에 퍼진 광풍에 가까운 '의대 블랙홀' 현상은 서울대도 피하지 못했습니다.서울대에선 최근 4년간 매년 10% 정도가 학교를 자퇴했는데 2019년 193명에서 2022년 328명으로 70% 가까이 증가했습니다.더욱이 이 기간, 의치대 자퇴생은 없는데 반해 공대는 328명, 농업생명과학대 277명, 자연과학대는 152명이나 학교를 그만 뒀습니다.이들 대부분이 N수로 의대에 진학했을 것으로 추정됩니다.[김원중 / 대성학원 입시전략실장 : 거의 압도적으로 의대를 가려고 하죠. (지방이라도? 서울의 좋은 대학교 돼도?) '지방이라도'가 아니라, 어디든 가죠. 대부분의 학생들 입장에서는 (의대와 다른 과들은) 비교 대상이 아니에요.][이만기 / 유웨이교육 평가연구소장 : (의대는) 수입이 되고 정년이 없는 것이 가장 강점인데, 이공계 학과에 가면 그걸 보장을 못 하잖아요.]의대 쏠림 현상은 심지어 같은 의약계열 내에서도 심각한 수준입니다.서울대에 합격하고도 다른 대학에 간 학생 비율을 보면, 1위부터 4위까지가 모두 의약계열인데, 서울대 치대조차 평균 3명 가운데 1명 넘게 등록을 포기했습니다.치과의사와 달리 일반 의사는 진료과목에 제한 없이 소위 '돈 되는 과'는 모두 개설

In [ ]:
df = pd.DataFrame(newsData, columns=["ranking", "title", "link", "img", "time", "contents"])


In [ ]:
df

,ranking,title,link,img,time,contents
0,1,"윤 대통령 ""공감해야 국민 통합""…통합위·여 지도부 만찬",https://n.news.naver.com/article/422/000062446...,https://mimgnews.pstatic.net/image/origin/422/...,2023-10-18 05:26:03,"윤석열 대통령은 ""국민 통합에는 전문성이 아닌 공감이 필요하다""고 강조했습니다.윤 ..."
1,2,강릉 급발진 사건 할머니 '혐의없음'…책임은 누가?,https://n.news.naver.com/article/422/000062452...,https://mimgnews.pstatic.net/image/origin/422/...,2023-10-18 13:22:04,[앵커]지난해 말 강원도 강릉에서 발생한 급발진 의심 사고로 초등학생 손자가 숨졌고...
2,3,'대통령실 앞 집회 금지 가능' 개정 집시법 시행,https://n.news.naver.com/article/422/000062448...,https://mimgnews.pstatic.net/image/origin/422/...,2023-10-18 07:49:01,앞으로 용산 대통령실 앞에서 경찰이 집회와 시위를 금지할 수 있게 됩니다.경찰에 따...
3,4,"[현장연결] ""안전한 나라""…윤대통령 '경찰의 날' 축사",https://n.news.naver.com/article/422/000062451...,https://mimgnews.pstatic.net/image/origin/422/...,2023-10-18 11:17:11,윤석열 대통령이 오는 21일 '경찰의 날'을 앞두고 열린 '제78주년 경찰의 날' ...
4,5,"""부축했다 성추행 몰려, 정학 부당""…대학생 승소",https://n.news.naver.com/article/422/000062453...,https://mimgnews.pstatic.net/image/origin/422/...,2023-10-18 13:31:01,술에 취한 동기 여학생을 부축했다 성추행으로 정학 처분을 받은 남학생이 대학 총장을...
5,1,"윤 대통령 ""경찰조직, 치안 중심으로 재편…저위험권총 등 신속 보급""",https://n.news.naver.com/article/437/000036362...,https://mimgnews.pstatic.net/image/origin/437/...,2023-10-18 11:21:01,"윤석열 대통령은 오늘(18일) 제78주년 경찰의 날 기념식에 참석해 ""경찰 조..."
6,2,"""의사형들 맛 좀 봐!"" 의대 증원 충돌 속 화제 된 변호사 글",https://n.news.naver.com/article/437/000036360...,https://mimgnews.pstatic.net/image/origin/437/...,2023-10-18 10:22:01,"[기자]의대 정원을 대폭 늘리는 방안을 놓고, 정부와 의사단체가 충돌하고 있습니다...."
7,3,"""쇼핑몰 문 잠궈!""…중국 관광객에 쇼핑 강요한 한국 여행사들",https://n.news.naver.com/article/437/000036360...,https://mimgnews.pstatic.net/image/origin/437/...,2023-10-18 10:03:01,"중국 단체 관광객들. 〈사진=연합뉴스〉 ""여기서 30분간 쇼핑하세요.""(..."
8,4,"마주 오는 얼굴이 '하나,둘,셋,넷…' 한문철도 ""이런 건 처음""",https://n.news.naver.com/article/437/000036361...,https://mimgnews.pstatic.net/image/origin/437/...,2023-10-18 10:49:01,킥보드 탄 여학생들 '아찔한 역주행'[기자]전북 전주시의 한 도로입니다.블랙박스 차...
9,5,"외교부, 일본 의원 야스쿠니 집단참배에 ""깊은 유감…역사 직시해야""",https://n.news.naver.com/article/437/000036361...,https://mimgnews.pstatic.net/image/origin/437/...,2023-10-18 10:36:01,일본 국회의원들이 오늘(18일) 오전 태평양전쟁 A급 전범이 합사된 야스쿠니 ...


In [ ]:
#01 코랩에서 한글 사용하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!pip install konlpy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (25.2 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120875 files and direc

In [ ]:
#코랩 matplotlib 한글 사용용
!pip install koreanize-matplotlib
import matplotlib.pyplot as plt
import koreanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.5 MB/s eta 0:00:00


In [ ]:
from wordcloud import WordCloud #워드클라우드 그리기
from konlpy.tag import Okt  #한글 형태소 분석
from collections import Counter  #단어 빈도수
import pandas as pd    #pandas
import matplotlib.pyplot as plt

In [ ]:
text=df['contents'] # contents 내용 추출하여 text에 저장

In [ ]:
text

0     윤석열 대통령은 "국민 통합에는 전문성이 아닌 공감이 필요하다"고 강조했습니다.윤 ...
1     [앵커]지난해 말 강원도 강릉에서 발생한 급발진 의심 사고로 초등학생 손자가 숨졌고...
2     앞으로 용산 대통령실 앞에서 경찰이 집회와 시위를 금지할 수 있게 됩니다.경찰에 따...
3     윤석열 대통령이 오는 21일 '경찰의 날'을 앞두고 열린 '제78주년 경찰의 날' ...
4     술에 취한 동기 여학생을 부축했다 성추행으로 정학 처분을 받은 남학생이 대학 총장을...
5        윤석열 대통령은 오늘(18일) 제78주년 경찰의 날 기념식에 참석해 "경찰 조...
6     [기자]의대 정원을 대폭 늘리는 방안을 놓고, 정부와 의사단체가 충돌하고 있습니다....
7        중국 단체 관광객들. 〈사진=연합뉴스〉    "여기서 30분간 쇼핑하세요."(...
8     킥보드 탄 여학생들 '아찔한 역주행'[기자]전북 전주시의 한 도로입니다.블랙박스 차...
9        일본 국회의원들이 오늘(18일) 오전 태평양전쟁 A급 전범이 합사된 야스쿠니 ...
10    윤석열 대통령이 17일 오후 청와대 영빈관에서 열린 국민통합위원회 만찬에 김한길 국...
11    더불어민주당 이재명 대표의 배우자 김혜경 씨가 연루된 '경기도 법인카드 사적 유용 ...
12     잇슈키워드, 시간입니다.첫 번째 키워드 보시죠.'노아의 방주'.성서에 나오는 얘기...
13    메리츠증권이 이화전기와 이아이디 등 이화그룹 계열 주식이 거래정지 되기 전 주식을 ...
14      [앵커] 기시다 후미오 일본 총리가 태평양전쟁 A급 전범이 합사된 야스쿠니 신사...
15    [레이더P] 제보자 조명현 씨 기자회견“李, 혈세 사적유용·절대권력자 행세”與장예찬...
16    [레이더P]한국도로공사 직원용 수영장이용률 저조 예상됐지만 개장aT, 임직원에 年1...
17    S60 한정판, 주저하니 품절‘텐미닛’ 완판 기록 또 달성가격은 변함없이 5

In [ ]:
change=list(map(str, text))

In [ ]:
text2=','.join(change)

In [ ]:
okt=Okt()
extract=okt.nouns(text2)

In [ ]:
extract

In [ ]:
# 명사 리스트에서 1글자 넘는 단어만 고르기
noun_list = []
for n in extract:
   if (len(n) >1):
     noun_list.append(n)

In [ ]:
noun_list

In [ ]:
counts=Counter(noun_list)

In [ ]:
tags=counts.most_common(100)  #빈도순으로 정리하기

In [ ]:
tags

In [ ]:
#리스트를 데이터프레임으로 변환하기
df=pd.DataFrame(tags, columns=['단어', '빈도'])
#원자료에서 상위 10개 행만 추출하기
df2=df.loc[0:9]

In [ ]:
df2

In [ ]:
df2.plot.bar(x='단어',y='빈도')

In [ ]:
import plotly.express as px
fig=px.bar(df2, x='빈도', y='단어', orientation='h')
fig.show()

In [ ]:
#워드 클라우드 생성하기
wc = WordCloud(font_path='/content/drive/MyDrive/2023년 2학기 데이터 강의 뉴스데이터 심화/NanumSquareEB__1_.ttf____',
               background_color="white",
               max_font_size=200, #가장 큰 글자 사이즈
               width = 600,  #그림가로길이
               height =600)  #그림 세로길이
cloud = wc.generate_from_frequencies(dict(tags))

#워드 클라우드 그리기
plt.figure(figsize=(8, 8)) #그림 사이즈
plt.axis('off') #x,y 축지정
plt.imshow(cloud)
plt.show()

In [ ]:
from PIL import Image
import numpy as np
# 마스크 이미지를 url에서 불러오기
img = Image.open('/content/drive/MyDrive/2023년 2학기 데이터 강의 뉴스데이터 심화/star.jpg____')    # 마스크가 될 이미지 불러오기
imgArray=np.array(img) #이미지 셀을 수치로 변환

In [ ]:
c2 = WordCloud(font_path='/content/drive/MyDrive/2023년 2학기 데이터 강의 뉴스데이터 심화/star.jpg____',
               background_color="white",
               max_font_size=200, #가장 큰 글자 사이즈
               width = 600,  #그림가로길이
               height =600,  #그림 세로길이
               mask=imgArray)

cloud2 = wc2.generate_from_frequencies(dict(tags))

#워드 클라우드 그리기
plt.figure(figsize=(8, 8)) #그림 사이즈
plt.axis('off') #x,y 축지정
plt.imshow(cloud2)
plt.show()

NameError: ignored